In [1]:
import pandas as pd
import numpy as np

# --- 1. Load Data ---
data = pd.read_excel(
    "ROUTPUTQvQd.xlsx",
    index_col = 0,
    na_values = ['#N/A'],
    keep_default_na = True
    )

lag = 4

# --- 2. Index & Column Cleanup ---
data.index.name = 'Date'
# Cleans the index format from 'YYYY:Q' to 'YYYYQ' and converts to PeriodIndex with quarterly frequency
data.index = data.index.astype(str).str.replace(':', '').map(lambda x: pd.Period(x, freq = 'Q'))

display(data)

# --- 3. Real-Time Calculation Function ---
## Calculates the real-time YoY growth rate by pulling both the first reported RGDP value
## and the revised RGDP value from the same vintage column.
def calculate_real_time_RGDP_yoy(df_levels, lag):
    real_time_vintage_cols = df_levels.apply(pd.Series.first_valid_index, axis = 1)
    real_time_yoy = pd.Series(index = df_levels.index, dtype = float)

    for obs_date in df_levels.index[lag:]:

        vintage_col_name = real_time_vintage_cols.loc[obs_date]

        if pd.isna(vintage_col_name):
            continue

        ## Get the full time time series of RGDP levels available on that trading date.
        vintage_series = df_levels[vintage_col_name]

        ## Exract numerator (t) and denomicator (t - 4) from the SAME vintage column.
        numerator_level = vintage_series.loc[obs_date]
        denominatorlevel = vintage_series.loc[obs_date - lag]
        if not pd.isna(numerator_level) and not pd.isna(denominatorlevel):
            yoy = ((numerator_level / denominatorlevel) -1) * 100
            real_time_yoy.loc[obs_date] = yoy

    return real_time_yoy

# --- 4. Execute All Necessary Calculations ---
final_revised_RGDP = data.iloc[:, -1]
final_revised_RGDP_yoy = ((final_revised_RGDP / final_revised_RGDP.shift(lag)) - 1) * 100
real_time_RGDP_yoy = calculate_real_time_RGDP_yoy(data, lag)
acceleration_signal = real_time_RGDP_yoy -  real_time_RGDP_yoy.shift(1)

# --- 5. Create Final Output DataFrame ---
final_output_df = pd.DataFrame({
    'Final_Revised_RGDP': final_revised_RGDP.values,
    'Final_Revised_RGDP_YoY_Growth': final_revised_RGDP_yoy.values,
    'Real_Time_YoY_Growth': real_time_RGDP_yoy,
    'Acceleration Signal': acceleration_signal
})

final_output_df.index.name = 'Date'
final_output_df = final_output_df.dropna().reset_index(drop = False).set_index('Date', drop = True)

display(final_output_df)

final_output_df.to_excel("Final_RGDP_YoY_Output(Q).xlsx")

,ROUTPUT65Q4,ROUTPUT66Q1,ROUTPUT66Q2,ROUTPUT66Q3,ROUTPUT66Q4,ROUTPUT67Q1,ROUTPUT67Q2,ROUTPUT67Q3,ROUTPUT67Q4,ROUTPUT68Q1,...,ROUTPUT23Q3,ROUTPUT23Q4,ROUTPUT24Q1,ROUTPUT24Q2,ROUTPUT24Q3,ROUTPUT24Q4,ROUTPUT25Q1,ROUTPUT25Q2,ROUTPUT25Q3,ROUTPUT25Q4
Date,,,,,,,,,,,,,,,,,,,,,
1947Q1,306.4,306.4,306.4,306.4,306.4,306.4,306.4,306.4,306.4,306.4,...,2034.5,2182.7,2182.7,2182.7,2182.7,2182.7,2182.7,2182.7,2182.7,2182.7
1947Q2,309.0,309.0,309.0,309.0,309.0,309.0,309.0,309.0,309.0,309.0,...,2029.0,2176.9,2176.9,2176.9,2176.9,2176.9,2176.9,2176.9,2176.9,2176.9
1947Q3,309.6,309.6,309.6,309.6,309.6,309.6,309.6,309.6,309.6,309.6,...,2024.8,2172.4,2172.4,2172.4,2172.4,2172.4,2172.4,2172.4,2172.4,2172.4
1947Q4,314.5,314.5,314.5,314.5,314.5,314.5,314.5,314.5,314.5,314.5,...,2056.5,2206.5,2206.5,2206.5,2206.5,2206.5,2206.5,2206.5,2206.5,2206.5
1948Q1,317.1,317.1,317.1,317.1,317.1,317.1,317.1,317.1,317.1,317.1,...,2087.4,2239.7,2239.7,2239.7,2239.7,2239.7,2239.7,2239.7,2239.7,2239.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024Q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,23386.2,23400.3,23400.3,23400.3,23478.6
2024Q4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23530.9,23542.3,23542.3,23586.5
2025Q1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23526.1,23512.7,23548.2


,Final_Revised_RGDP,Final_Revised_RGDP_YoY_Growth,Real_Time_YoY_Growth,Acceleration Signal
Date,,,,
1948Q2,2276.7,4.584501,4.498382,1.006215
1948Q3,2289.8,5.404161,5.232558,0.734176
1948Q4,2292.4,3.893043,4.515103,-0.717455
1949Q1,2260.8,0.942090,2.333649,-2.181455
1949Q2,2253.1,-1.036588,-0.123877,-2.457526
...,...,...,...,...
2024Q2,23286.5,3.126592,3.119404,0.150016
2024Q3,23478.6,2.791471,2.657050,-0.462354
2024Q4,23586.5,2.399517,2.483820,-0.173230
